In [1]:
dataset_folder_name = 'TrainFiles/Splits/FiftyPercentSplit'
testset_folder_name_WM = 'TestFiles/WM'
testset_folder_name_WW = 'TestFiles/WW'
testset_folder_name_BM = 'TestFiles/BM'
testset_folder_name_BW = 'TestFiles/BW'

TRAIN_TEST_SPLIT = 0.7
IM_WIDTH = IM_HEIGHT = 198

dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((g, i) for i, g in dataset_dict['race_id'].items())
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Parsing

In [2]:
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            filename = os.path.split(path)[1]
            filename = os.path.splitext(filename)[0]
            age, gender, race, _ = filename.split('_')

            return dataset_dict['race_id'][int(race)]
        except Exception as ex:
            return None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    random.shuffle(files)
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['race', 'file']
    df = df.dropna()
    
    return df

In [3]:
df = parse_dataset(dataset_folder_name)
testset_WM = parse_dataset(testset_folder_name_WM)
testset_WW = parse_dataset(testset_folder_name_WW)
testset_BM = parse_dataset(testset_folder_name_BM)
testset_BW = parse_dataset(testset_folder_name_BW)
testset_WM.head()

,race,file
0,white,TestFiles/WM/79_0_0_20170111223823788.jpg.chip...
1,white,TestFiles/WM/9_0_0_20170110215516060.jpg.chip.jpg
2,white,TestFiles/WM/26_0_0_20170116181255495.jpg.chip...
3,white,TestFiles/WM/67_0_0_20170104213103526.jpg.chip...
4,white,TestFiles/WM/60_0_0_20170117173825225.jpg.chip...


# Makes Data Generator For Trainset

In [4]:
from keras.utils import to_categorical
from PIL import Image


class UtkFaceDataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self, SPLIT):
        p = np.random.permutation(len(self.df))
        train_up_to = int(len(self.df) * SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]

        train_up_to = int(train_up_to * SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        # converts alias to id
        self.df['race_id'] = self.df['race'].map(lambda gender: dataset_dict['race_alias'][gender])
        
        
        return train_idx, valid_idx, test_idx
    
    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        im = Image.open(img_path)
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        images, races = [], []
        while True:
            for idx in image_idx:
                person = self.df.iloc[idx]
                
                race = person['race_id']
                file = person['file']
                
                im = self.preprocess_image(file)
                races.append(to_categorical(race, len(dataset_dict['race_id'])))
                images.append(im)
                
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(races)]
                    images, races, = [], []
                    
            if not is_training:
                break
                
data_generator = UtkFaceDataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes(TRAIN_TEST_SPLIT)

Using TensorFlow backend.


# Data Generator TestSet

In [5]:
TEST_DATA_SPLIT = 0
#WM
test_data_generator_WM = UtkFaceDataGenerator(testset_WM)
train_idx_test_WM, valid_idx_test_WM, test_idx_test_WM = test_data_generator_WM.generate_split_indexes(TEST_DATA_SPLIT)
#WW
test_data_generator_WW = UtkFaceDataGenerator(testset_WW)
train_idx_test_WW, valid_idx_test_WW, test_idx_test_WW = test_data_generator_WW.generate_split_indexes(TEST_DATA_SPLIT)
#BM
test_data_generator_BM = UtkFaceDataGenerator(testset_BM)
train_idx_test_BM, valid_idx_test_BM, test_idx_test_BM = test_data_generator_BM.generate_split_indexes(TEST_DATA_SPLIT)
#BW
test_data_generator_BW = UtkFaceDataGenerator(testset_BW)
train_idx_test_BW, valid_idx_test_BW, test_idx_test_BW = test_data_generator_BW.generate_split_indexes(TEST_DATA_SPLIT)






# # Making Model

In [6]:
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf

class UtkMultiOutputModel():
    """
    Used to generate our multi-output model. This CNN contains three branches, one for age, other for 
    sex and another for race. Each branch contains a sequence of Convolutional Layers that is defined
    on the make_default_hidden_layers method.
    """
    def make_default_hidden_layers(self, inputs):
        """
        Used to generate a default set of hidden layers. The structure used in this network is defined as:
        
        Conv2D -> BatchNormalization -> Pooling -> Dropout
        """
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)

        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)

        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)

        return x

    def build_race_branch(self, inputs, num_races):
        """
        Used to build the race branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)

        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_races)(x)
        x = Activation("softmax", name="race_output")(x)

        return x

    def build_gender_branch(self, inputs, num_genders=2):
        """
        Used to build the gender branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)

        x = self.make_default_hidden_layers(inputs)

        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_genders)(x)
        x = Activation("sigmoid", name="gender_output")(x)

        return x

    def build_age_branch(self, inputs):   
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.

        """
        x = self.make_default_hidden_layers(inputs)

        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="age_output")(x)

        return x

    def assemble_full_model(self, width, height, num_races):
        """
        Used to assemble our multi-output model CNN.
        """
        input_shape = (height, width, 3)

        inputs = Input(shape=input_shape)

        race_branch = self.build_race_branch(inputs, num_races)

        model = Model(inputs=inputs,
                     outputs = [race_branch],
                     name="face_net")

        return model
    
model = UtkMultiOutputModel().assemble_full_model(IM_WIDTH, IM_HEIGHT, num_races=len(dataset_dict['race_alias']))













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
'''%matplotlib inline

from keras.utils import plot_model
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

plot_model(model, to_file='model.png')
img = mpimg.imread('model.png')

plt.figure(figsize=(40, 30))
plt.imshow(img)'''

"%matplotlib inline\n\nfrom keras.utils import plot_model\nimport matplotlib.image as mpimg\nimport matplotlib.pyplot as plt\n\nplot_model(model, to_file='model.png')\nimg = mpimg.imread('model.png')\n\nplt.figure(figsize=(40, 30))\nplt.imshow(img)"

# Training

In [8]:
from keras.optimizers import Adam

init_lr = 1e-4
epochs = 100

opt = Adam(lr=init_lr, decay=init_lr / epochs)

model.compile(optimizer=opt, 
              loss={
                  'race_output': 'categorical_crossentropy'
                  },
              loss_weights={ 
                  'race_output': 1.5},
              metrics={
                  'race_output': 'accuracy',})

valid_batch_size = 32
print(len(valid_idx))
print(len(valid_idx)//valid_batch_size)



1519
47


And now let's train our model with a batch size of 32 for both valid and train sets. We will be using a ModelCheckpoint callback in order to save the model on disk at the end of each epoch.

In [9]:
from keras.callbacks import ModelCheckpoint

batch_size = 32
valid_batch_size = 32
train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)

callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]

history = model.fit_generator(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
110/110 [==============================] - 14s 132ms/step - loss: 1.1270 - acc: 0.7321 - val_loss: 0.8310 - val_acc: 0.7600
Epoch 2/100
110/110 [==============================] - 9s 79ms/step - loss: 0.8934 - acc: 0.7835 - val_loss: 0.8566 - val_acc: 0.7354
Epoch 3/100
110/110 [==============================] - 9s 79ms/step - loss: 0.6896 - acc: 0.8315 - val_loss: 1.0027 - val_acc: 0.7334
Epoch 4/100
110/110 [==============================] - 9s 79ms/step - loss: 0.6486 - acc: 0.8386 - val_loss: 1.3383 - val_acc: 0.7473
Epoch 5/100
110/110 [==============================] - 9s 80ms/step - loss: 0.5557 - acc: 0.8648 - val_loss: 1.6068 - val_acc: 0.7314
Epoch 6/100
110/110 [==============================] - 9s 79ms/step - loss: 0.5140 - acc: 0.8733 - val_loss: 1.5977 - val_acc: 0.7646
Epoch 7/100
110/110 [==============================] - 9s 79ms/step - loss: 0.5481 - acc: 0.8716 -

110/110 [==============================] - 9s 82ms/step - loss: 0.1096 - acc: 0.9702 - val_loss: 0.6478 - val_acc: 0.8850
Epoch 57/100
110/110 [==============================] - 9s 78ms/step - loss: 0.1145 - acc: 0.9739 - val_loss: 0.5193 - val_acc: 0.8996
Epoch 58/100
110/110 [==============================] - 9s 79ms/step - loss: 0.1045 - acc: 0.9713 - val_loss: 0.5517 - val_acc: 0.8969
Epoch 59/100
110/110 [==============================] - 9s 78ms/step - loss: 0.0944 - acc: 0.9787 - val_loss: 0.5856 - val_acc: 0.8883
Epoch 60/100
110/110 [==============================] - 9s 78ms/step - loss: 0.0952 - acc: 0.9770 - val_loss: 0.5231 - val_acc: 0.8989
Epoch 61/100
110/110 [==============================] - 9s 78ms/step - loss: 0.1065 - acc: 0.9739 - val_loss: 0.5430 - val_acc: 0.9043
Epoch 62/100
110/110 [==============================] - 9s 79ms/step - loss: 0.0945 - acc: 0.9787 - val_loss: 0.5068 - val_acc: 0.9023
Epoch 63/100
110/110 [==============================] - 9s 78ms/step

In [10]:
import plotly.graph_objects as go
from IPython.display import display, Image

# Testing

In [11]:
from PIL import Image

test_batch_size = 128


#### White Men

In [12]:

test_generator = test_data_generator_WM.generate_images(test_idx_test_WM, is_training=False, batch_size=test_batch_size)
race_pred = model.predict_generator(test_generator, steps=len(test_idx_test_WM)//test_batch_size)



test_generator = test_data_generator_WM.generate_images(test_idx_test_WM, is_training=False, batch_size=test_batch_size)
samples = 0
images, race_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    race_true.extend(labels[0])


race_true = np.array(race_true)
race_true = race_true.argmax(axis=-1)
race_pred = race_pred.argmax(axis=-1)




from sklearn.metrics import classification_report

cr_race = classification_report(race_true, race_pred, target_names=dataset_dict['race_alias'].keys())
print(cr_race)


              precision    recall  f1-score   support

       white       1.00      0.95      0.98       384
       black       0.00      0.00      0.00         0

    accuracy                           0.95       384
   macro avg       0.50      0.48      0.49       384
weighted avg       1.00      0.95      0.98       384



/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



#### White Women

In [13]:

test_generator = test_data_generator_WW.generate_images(test_idx_test_WW, is_training=False, batch_size=test_batch_size)
race_pred = model.predict_generator(test_generator, steps=len(test_idx_test_WW)//test_batch_size)



test_generator = test_data_generator_WW.generate_images(test_idx_test_WW, is_training=False, batch_size=test_batch_size)
samples = 0
images, race_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    race_true.extend(labels[0])


race_true = np.array(race_true)
race_true = race_true.argmax(axis=-1)
race_pred = race_pred.argmax(axis=-1)




from sklearn.metrics import classification_report

cr_race = classification_report(race_true, race_pred, target_names=dataset_dict['race_alias'].keys())
print(cr_race)

              precision    recall  f1-score   support

       white       1.00      0.97      0.98       384
       black       0.00      0.00      0.00         0

    accuracy                           0.97       384
   macro avg       0.50      0.48      0.49       384
weighted avg       1.00      0.97      0.98       384



/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



#### Black Men

In [14]:

test_generator = test_data_generator_BM.generate_images(test_idx_test_BM, is_training=False, batch_size=test_batch_size)
race_pred = model.predict_generator(test_generator, steps=len(test_idx_test_BM)//test_batch_size)



test_generator = test_data_generator_BM.generate_images(test_idx_test_BM, is_training=False, batch_size=test_batch_size)
samples = 0
images, race_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    race_true.extend(labels[0])


race_true = np.array(race_true)
race_true = race_true.argmax(axis=-1)
race_pred = race_pred.argmax(axis=-1)




from sklearn.metrics import classification_report

cr_race = classification_report(race_true, race_pred, target_names=dataset_dict['race_alias'].keys())
print(cr_race)


              precision    recall  f1-score   support

       white       0.00      0.00      0.00         0
       black       1.00      0.89      0.94       384

    accuracy                           0.89       384
   macro avg       0.50      0.45      0.47       384
weighted avg       1.00      0.89      0.94       384



/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



#### Black Women

In [15]:

test_generator = test_data_generator_BW.generate_images(test_idx_test_BW, is_training=False, batch_size=test_batch_size)
race_pred = model.predict_generator(test_generator, steps=len(test_idx_test_BW)//test_batch_size)



test_generator = test_data_generator_BW.generate_images(test_idx_test_BW, is_training=False, batch_size=test_batch_size)
samples = 0
images, race_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    race_true.extend(labels[0])


race_true = np.array(race_true)
race_true = race_true.argmax(axis=-1)
race_pred = race_pred.argmax(axis=-1)




from sklearn.metrics import classification_report

cr_race = classification_report(race_true, race_pred, target_names=dataset_dict['race_alias'].keys())
print(cr_race)


              precision    recall  f1-score   support

       white       0.00      0.00      0.00         0
       black       1.00      0.91      0.95       384

    accuracy                           0.91       384
   macro avg       0.50      0.45      0.48       384
weighted avg       1.00      0.91      0.95       384



/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



### Example of predictions